# Challenge: Machine Learning for Drone Identification
In this Jupyter notebook are contained the short answers/invocations of appropriate code in response to the assignment components of the challenge in question. However, most of the long-winded implementation is hidden away in `.py` files we wrote, that are imported here. Thus, for this to work, it is recommended you clone the entirety of [our associated Github repo](https://github.com/DanielShteinbok/CANSOFCOM_Drone_2021), and opening the Jupyter notebook contained within that (it's this one, but its location within the repo simply ensures that local imports work properly with minimal troubleshooting).

**Additional dependencies** are as follows:
* `scipy`
* `numpy`
* `matplotlib`

**TODO:** add further dependencies for ML...


In [8]:
# This is our implementation of the signal returned from the drone
import drone_signal

# See additional dependencies:
import scipy
import numpy
import matplotlib

## 1. Implement Time-Domain Radar Return Signals from Drones
The actual implementation for this is contained within the function `drone_signal.return_signal_array`. The variables we chose as representative values are set in a code block below, and the meanings are as follow:


`Ar` is $A_r$ in the equation, the real scale factor,

`L1` is $L_1$ in the equation, the distance of the blade roots from the centre of rotation in metres,

`L2` is $L_2$ in the equation, the distance of the blade tips from the centre of rotation in metres,

`N` is $N$ in the equation, number of blades,

`R` is $R$ in the equation, range of the centre of rotation (distance from the radar receiver) in metres,

`Vrad` is $V_{rad}$ in the equation, radial velocity in metres per second,

`lam` is $\lambda$ in the equation, the wavelength of the transmitted signal in metres,

`theta` is $\theta$ in the equation, the angle between the plane of rotation and the line of sight from the radar to the centre of rotation, in radians

`fc` is $f_c$ in the equation, the transmitted frequency (assumed to be $f_c = \frac{c}{\lambda}$, where $c$ is the speed of light

`frot` is $f_{rot}$ in the equation, the frequency of rotation in Hertz

In [9]:
Ar = 1
L1 = 1
L2 = 0
N = 4
R = 500
Vrad = 1

theta = scipy.pi/2 #depression angle, say 30deg
fc = 60e9 #60 GHz
lam = 299792458/fc #wavelength wavelength = c / frequency
fs = 1000 #sampling frequency
frot = 60 #frequency of drone prop rotation

The next step is to create the numpy array of the times at which the returning frequency is sampled: